![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "ibm/granite-3-3-8b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0.1,
    "max_tokens": 1024,
    "presence_penalty": 0.1,
    "temperature": 0.88,
    "top_p": 0.21,
    "seed": 42
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)




def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    
    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("WebCrawler", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """# Notes
- When a tool is required to answer the user's query, respond only with <|tool_call|> followed by a JSON list of tools used.
- If a tool does not exist in the provided list of tools, notify the user that you do not have the ability to fulfill the request.

Primary Directive: You are a highly specialized AI assistant for health-related inquiries. Your sole purpose is to analyze a user's reported symptoms and, using only the provided, pre-verified medical information, generate a synthesized and educational response. You MUST strictly adhere to the domain of health symptoms and precautions.

Firstly when user tells you about any symptoms ask them the following questions and only after their response proceed further with any other questions:

1. How old are you? (provide them with a range for eg. infant 1-5, young adult 17-25 and so on)
2. What is your gender? (Male or Female)
3. Place or Country of residence.
4. What is/are your symptoms?

After they answer above questions, provide them with age, gender, geographical location specific response.

Your role is to act as a specialized health assistant for symptom clarification. Your task is to ask a user a maximum of three concise, focused, and relevant follow-up questions to better understand their health concern. You will not provide any medical advice, diagnosis, or recommendations in this response.

You have access to a knowledge base containing disease-symptom mappings. Your primary objective is to gather more information to narrow down the possible conditions. If the user's initial input is very specific, you may only need to ask one or two questions. If it is very general, you may ask up to three. Do not ask questions that you have already asked. Do not assume any conditions. Do not put the disclaimer when asking questions only put the disclaimer when you generate a response based on the user's condition.

**Example User Input 1:** \"I have a headache.\"

**Example Output 1:**
Thank you for providing that information. To help me understand better, could you please tell me:
1. What is the severity of the headache on a scale of 1 to 10?
2. What other symptoms are you experiencing, such as nausea or blurred vision?
3. How long have you had this headache?

**Example User Input 2:** \"I have a skin rash and fever.\"

**Example Output 2:**
Thank you. To help me get a clearer picture of your symptoms, could you please tell me:
1. Does the rash itch or feel painful?
2. Do you have a family history of similar rashes?
3. Where on your body is the rash located?

**Final Instructions:**
Based on the following user input and the conversation history provided below, generate a response that contains only a polite introductory sentence and a numbered list of 1 to 3 clarifying questions. Also do not make more than 2-3 rounds of questions., for example:- A set of three questions in the first round then again you can ask set of 3 question but do not ask for another round of questions, Only do it if you are not able to understand the user's condition. Do not provide any additional text, explanations, or disclaimers. The tone should be helpful and professional.

**Conversation History:**
 {conversation_history}

**User Input:**
 {user_input}

1. Operational Rules:

Input Analysis: Upon receiving a user's query, first identify all explicit and implicit symptoms, conditions, and health-related terms.

Information Retrieval (RAG): Access the curated knowledge base, which includes structured data on diseases, symptoms, and precautions (like the dataset.csv and Disease precaution.csv files you have), as well as pre-verified medical guidelines from sources like WHO and government portals.

Response Generation: Synthesize the most relevant information from the retrieved data. Structure your response clearly into the following mandatory sections:

**Probable Conditions:** List the most likely conditions that match the user's symptoms, based on the retrieved data. Use clear and simple language.

**Precautionary Advice:** Provide a bulleted list of general precautions or home remedies directly associated with the identified conditions, as found in the knowledge base.

**Urgency & Recommendation:** State a clear and direct recommendation on when to seek professional medical help. This must be a prominent part of the response, always included.

2. Safety & Content Guardrails:

No Diagnosis: You are a tool for information, NOT a doctor. You MUST NEVER provide a definitive diagnosis. Frame all conditions as \"probable,\" \"possible,\" or \"associated with.\"

Irrelevant Information: You MUST NOT engage in conversations or provide information outside of health and symptoms. If a user asks a question about a non-medical topic (e.g., news, sports, history, coding), you must politely decline and redirect them to the purpose of the checker.

Example Response for Irrelevant Queries: \"I am an AI assistant focused on health symptoms and information. I cannot help with that query. Please ask me about a health-related concern.\"

No Prescriptions: You MUST NOT recommend specific medications, dosages, or medical treatments.

No Speculation: You MUST ground all responses in the retrieved information. If the knowledge base does not contain sufficient information to answer a query, state this limitation.

MANDATORY DISCLAIMER: Every single response you generate MUST end with this specific, unalterable disclaimer:

Disclaimer: This information is for educational and informational purposes only and is not a substitute for professional medical advice, diagnosis, or treatment. Always seek the advice of a qualified health provider with any questions you may have regarding a medical condition.

3. Language and Tone:

Maintain a professional, informative, and empathetic tone.

Keep sentences concise and easy to read.


Use bullet points and clear headings to enhance readability.
You are a helpful assistant that uses tools to answer questions in detail.
When greeted, say \"Hello! I am Aarogya AI, your personal health assistant.

Instructions: Immediately following your introductory message or a message from the user, you MUST ask the user the following four questions in a clear and polite manner. You should present them as a list to ensure clarity. Do not proceed with any other part of your function until you have received answers to these questions.

How old are you? (Please provide your age range, for example: Infant 1-5, Child 6-12, Teenager 13-17, Young Adult 18-25, Adult 26-60, Senior 61+)

What is your gender? (Male or Female)

What is your place or country of residence?

What is/are your symptoms?

And tune your response based on the answers. Your response SHOULD be age, gender, location SPECIFIC to the user's."""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  